<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/notebooks/StyleGAN_Paintings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Originally taken from https://colab.research.google.com/drive/1cFKK0CBnev2BF8z9BOHxePk7E-f7TtUi
which was foind on this Reddit post by user _C0D32_:
https://www.reddit.com/r/MachineLearning/comments/bagnq6/p_stylegan_trained_on_paintings_512x512/

Other neat resources pointed out in that reddit post are:
- Sample of 999 generated images (512x512): https://imgur.com/a/8nkMmeB
- Training data based on (only took images >= 1024x1024 (~30k)): https://www.kaggle.com/c/painter-by-numbers/data
- quick latent space interpolation between 2 random vectors: https://imgur.com/a/VXt0Fhs
- trained model: https://mega.nz/#!PsIQAYyD!g1No7FDZngIsYjavOvwxRG2Myyw1n5_U9CCpsWzQpIo

In [0]:
!git clone https://github.com/NVlabs/stylegan

In [0]:
%cd stylegan

In [0]:
!wget https://github.com/parameter-pollution/stylegan_paintings/releases/download/v0.1/network-snapshot-008040.pkl

In [0]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

In [0]:
tflib.init_tf()

In [0]:
model_path = "./network-snapshot-008040.pkl"

In [0]:
with open(model_path,"rb") as f:
        _G, _D, Gs = pickle.load(f)

In [0]:
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

Set the random number seed

In [0]:
# change this number to get a different image 
rnd = np.random.RandomState(42)

In [0]:
#rerun this line to get a different image
latent_vector1 = rnd.randn(1, Gs.input_shape[1])

In [0]:
images = Gs.run(latent_vector1, None, truncation_psi=1, randomize_noise=False, output_transform=fmt)
PIL.Image.fromarray(images[0])

Modifications by Kevin Sikorski below this section

They don't work yet.

In [0]:
#rerun this line to get a different image
latent_vector1 = rnd.randn(1, Gs.input_shape[1])
images = Gs.run(latent_vector1, None, truncation_psi=1, randomize_noise=False, output_transform=fmt)
PIL.Image.fromarray(images[0])

In [0]:
print(latent_vector1.shape)
print(latent_vector1)

In [0]:
! git clone https://github.com/iyaja/stylegan-encoder.git
import os
os.chdir("stylegan-encoder")

import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

URL_FFHQ = 'https://s3-us-west-2.amazonaws.com/nanonets/blogs/karras2019stylegan-ffhq-1024x1024.pkl'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs = pickle.load(f)

generator = Generator(Gs, batch_size=1, randomize_noise=False)

In [0]:

duration_sec = 5.0
smoothing_sec = 1.0
mp4_fps = 20
num_frames = int(np.rint(duration_sec * mp4_fps))

def generate_image_for_video(latent_vector):
    #latent_vector = latent_vector.reshape((1, 18, 512))
    latent_vector = latent_vector.reshape((1, 1, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]

    return img_array
  
  
def move_for_video(latent_vector, direction, coeff):
  
  new_latent_vector = latent_vector.copy()
  new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
  
  img_array = generate_image(new_latent_vector)
  
  return img_array


# If you're interpolating between two characters, set these characters here
char1 = rnd.randn(1, Gs.input_shape[1])  #daenerys
char2 = rnd.randn(1, Gs.input_shape[1])  #jaime

# This creates an nd array that stores all the image frames fot cross-character interpolation
src_images = np.stack(generate_image_for_video((0.01*alpha*char2)+((1-(0.01*alpha))*char1)) for alpha in range (100))


# Uncomment the next line if you want to do a character transforation video, and choose the arguments as per your requirement
#src_images = np.stack(move_for_video(dany_meme, smile_direction, (0.02*alpha)) for alpha in range (-100,100))


def make_frame(t):
    frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
    src_image = src_images[frame_idx]
    return np.array(src_image)

# Generate video.
import moviepy.editor
mp4_file = 'results/interpolate.mp4'
mp4_codec = 'libx264'
mp4_bitrate = '5M'

video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)